### Example of DNNs in regression - Housing dataset

Loading data

In [1]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

57026/57026 [==============================] - 0s 2us/step
(404, 13)
(102, 13)
50.0 5.0


Standardizing data

In [2]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

Building the model (deine function to enable more flexibility in tests)

In [3]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model(hidden = 64):
    model = models.Sequential()
    model.add(layers.Dense(hidden, activation='relu',
                            input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(hidden, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [4]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=1)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)

2024-02-21 17:00:22.926329: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-21 17:00:22.926353: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-21 17:00:22.926360: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-21 17:00:22.926971: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-21 17:00:22.927444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/80


2024-02-21 17:00:23.495304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


26/26 [==============================] - 2s 53ms/step - loss: 562.2321 - mae: 22.0913
Epoch 2/80
26/26 [==============================] - 0s 10ms/step - loss: 523.7892 - mae: 21.5851
Epoch 3/80
26/26 [==============================] - 0s 10ms/step - loss: 497.8592 - mae: 21.1221
Epoch 4/80
26/26 [==============================] - 0s 10ms/step - loss: 473.6342 - mae: 20.5058
Epoch 5/80
26/26 [==============================] - 0s 11ms/step - loss: 459.3235 - mae: 20.0476
Epoch 6/80
26/26 [==============================] - 0s 11ms/step - loss: 427.7131 - mae: 19.3648
Epoch 7/80
26/26 [==============================] - 0s 10ms/step - loss: 400.7655 - mae: 18.7790
Epoch 8/80
26/26 [==============================] - 0s 10ms/step - loss: 375.0630 - mae: 18.0477
Epoch 9/80
26/26 [==============================] - 0s 10ms/step - loss: 338.0024 - mae: 16.9243
Epoch 10/80
26/26 [==============================] - 0s 10ms/step - loss: 323.9745 - mae: 16.5464
Epoch 11/80
26/26 [=====================

2024-02-21 17:00:45.863601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 191ms/step - loss: 26.2949 - mae: 4.2410
26.294906616210938 4.240991592407227


In [5]:
import numpy as np
k = 5
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []
model = build_model()
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    partial_train_data = np.concatenate( [train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print(all_scores)
print(np.mean(all_scores))

processing fold # 0


2024-02-21 17:00:46.754246: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-21 17:04:43.713669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
[3.6398773193359375, 4.735353946685791, 3.5560965538024902, 4.670096397399902, 12.230127334594727]
5.766310310363769
